In [1]:
import pandas as pd
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict

/usr/local/lib/python3.5/dist-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('seekingAlpha_transcript_single_page_20180710.txt', delimiter='|')

In [3]:
df.columns

Index(['input company name', 'About', 'Transcript page heading',
       'Transcript heading', 'Date published', 'Preamble',
       'transcript after preamble', 'Transcript link', 'Scraped date'],
      dtype='object')

In [4]:
trascripts = df['transcript after preamble']

In [5]:
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict

In [6]:
def get_analysts_from_preamble(preamble):
    if not preamble:
        return
    try:
        lines_raw = list(map(lambda x: x.strip(), preamble.split('^')))
        lines = list(zip(range(len(lines_raw)), lines_raw))
        B_index = list(map(lambda x: x[0], filter(lambda x: x[1].startswith('~'), lines)))
        B_index += [len(lines)]
#         print(B_index, lines)
        executive_index = list(filter(lambda x: x[1].lower().startswith('~executive'), lines))[0][0]
        executive_index_range = executive_index, B_index[B_index.index(executive_index)+1]
        executives_lines = lines_raw[executive_index_range[0]+1: executive_index_range[1]]
        executives = list(map(lambda x: x.split('-')[0].strip(), executives_lines))
        analysts_index = list(filter(lambda x: x[1].lower().startswith('~analyst'), lines))[0][0]
        analysts_index_range = analysts_index, B_index[B_index.index(analysts_index)+1]
        analysts_lines = lines_raw[analysts_index_range[0]+1: analysts_index_range[1]]
        analysts = list(map(lambda x: x.split('-')[0].strip(), analysts_lines))
        
    except Exception as e:
#         print(e, traceback.format_exc())
#         print('PREAMBLE', preamble)
#         print('-------------------------------')
        return None 
    
    
#     print(B_index, executive_index, analysts_index)
    
    
    return executives, analysts

In [7]:
import gensim

In [8]:
# df[df['executives_analysts'].isnull()]['Preamble']

In [9]:
df.Preamble = df.Preamble.fillna('')
df['executives_analysts'] = df.Preamble.apply(get_analysts_from_preamble)

In [10]:
def get_q_a(heading, transcript, executives_analysts):
    executives = executives_analysts[0]
    analysts = executives_analysts[1]
    def if_analyst(line):
        if not line.startswith('~'):
            return False
#         name = B.strip('|B').strip().split('-')[0].strip()
        for analyst in analysts:
            if fuzz.partial_ratio(line, analyst) > 93:
#                 print(line, analyst)
                return analyst
        return False
    
    def if_executive(line):
        if not line.startswith('~'):
            return False
#         name = B.strip('|B').strip().split('-')[0].strip()
        for executive in executives:
            if fuzz.partial_ratio(line, executive) > 93:
                return executive
        return False
    
    def return_name(B):
        name = process.extractOne(B, executives + analysts)[0][0]
#         return name[0]
        if fuzz.partial_ratio(B, name) > 0.90:
            return name
        else:
            return 
        
    
    lines_raw = list(map(lambda x: x.strip(), transcript.split('^')))
    lines = []
    for i in range(len(lines_raw)):
        if if_analyst(lines_raw[i]):
            lines.append(('analyst', if_analyst(lines_raw[i]), lines_raw[i+1]))
            i = i + 2
        elif if_executive(lines_raw[i]):
            lines.append(('executive', if_executive(lines_raw[i]), lines_raw[i+1]))
            i = i + 2
        else:
            i = i + 1 
        if i >= len(lines_raw):
            break

    q_a = defaultdict(list)
    for line in lines:
        if line[0] == 'analyst':
            q = line
        elif line[0] == 'executive':
            q_a[q].append(line)
    q_a['heading'] = heading 
    return q_a

In [11]:
data = {}
for _, row in df.iterrows():
#     print(row['Transcript page heading'])
    try:
        q_a = get_q_a(row['Transcript page heading'], row['transcript after preamble'], row['executives_analysts'])
    #     print q_a
        data.update({row['Transcript page heading']: q_a})
#         print q_a
    except:
        pass
#         print(row)
#         print('---------------')

### Filter OUT 

In [74]:
aapl_keys = list(filter(lambda x: x.lower().find('aapl') != -1, data.keys()))

In [83]:
data_subset = {aapl_key: data[aapl_key] for aapl_key in aapl_keys}
data_old, data = data, data_subset

In [87]:
q_a = [(ques[2], ans) for trans in data.values() for ques, ans in trans.items()]

questions = [ques[2] for trans in data.values() for ques in trans.keys()]

In [88]:
# data_subset?

In [144]:
# q_a[0]

# import pickle
# pickle.dump(data, open('data_q_a.pic', 'wb'))

# import pandas

pandas.DataFrame(list(zip(questions, list(map(lambda x: list(nlp(x).noun_chunks), questions)))), columns=['questions', 'noun_chunks'])#.to_csv('question.csv')

In [131]:
%autoreload 2
import spacy

UsageError: Line magic function `%autoreload` not found.


In [132]:
# !python -m spacy download en 

In [148]:
nlp.vocab['you'].is_stop

True

In [136]:
from spacy.en import English
nlp = English()

ImportError: No module named 'spacy.en'

In [137]:
nlp = spacy.load('en')

In [182]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english') + ['thanks', 'u'])
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    doc = str(doc)
        
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    
    return normalized
questions_cleaned = [clean(ques).split() for ques in questions]    

In [168]:
df = pandas.DataFrame(list(zip(questions, list(map(lambda x: list(nlp(x).noun_chunks), questions)))), columns=['questions', 'noun_chunks'])

In [176]:
nlp.vocab['thanks'].is_stop = True

In [180]:
# a = df['noun_chunks'][0]
# # list(map(clean, a))
# a =a[0]
str(a)

'you'

In [184]:
df['noun_chunks'] = df['noun_chunks'].apply(lambda x: list(map(clean, x)))

In [185]:
df.to_csv('question_w_np.csv')

In [112]:
clean(questions[0])

'great okay thank you'

In [107]:
clean("thank you")

'thank'

In [108]:
questions_cleaned[:10]

[['great', 'okay', 'thank', 'you'],
 ['hey',
  'good',
  'afternoon',
  'congratulation',
  'tim',
  'were',
  'month',
  'iphone',
  '7',
  'seeing',
  'anything',
  'measurable',
  'term',
  'growing',
  'trend',
  'annual',
  'upgrade',
  'second',
  'is',
  'historically',
  'term',
  'new',
  'product',
  'category',
  'guy',
  'always',
  'looked',
  'unique',
  'advantage',
  'getting',
  'segment',
  'im',
  'curious',
  'car',
  'lot',
  'rumor',
  'there',
  'would',
  'like',
  'perspective',
  'think',
  'advantage',
  'apple',
  'could',
  'add',
  'auto',
  'space'],
 ['thank',
  'you',
  'tim',
  'weve',
  'seen',
  'increasing',
  'focus',
  'artificial',
  'intelligence',
  'smartphones',
  'like',
  'new',
  'pixel',
  'google',
  'also',
  'home',
  'assistance',
  'like',
  'amazon',
  'echo',
  'guy',
  'obviously',
  'siri',
  'well',
  'curious',
  'think',
  'balancing',
  'ai',
  'focus',
  'privacy',
  'also',
  'important',
  'dedicated',
  'home',
  'assista

In [97]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(questions_cleaned)
doc_term_matrix = [dictionary.doc2bow(question) for question in questions_cleaned]
lda = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

In [96]:
# questions

In [94]:
lda.print_topics(num_topics=190, num_words=10)

[(0,
  '0.026*"inventory" + 0.026*"term" + 0.026*"apple" + 0.026*"quarter" + 0.017*"think" + 0.017*"followup" + 0.017*"talk" + 0.017*"channel" + 0.017*"supply" + 0.017*"seeing"'),
 (1,
  '0.025*"think" + 0.025*"quarter" + 0.025*"one" + 0.025*"tim" + 0.025*"general" + 0.017*"you" + 0.017*"last" + 0.017*"up" + 0.017*"maybe" + 0.017*"understand"'),
 (2,
  '0.055*"there" + 0.037*"tim" + 0.037*"market" + 0.037*"million" + 0.018*"you" + 0.018*"thank" + 0.018*"time" + 0.018*"excited" + 0.018*"manufacturing" + 0.018*"establishing"'),
 (3,
  '0.037*"kind" + 0.037*"maybe" + 0.028*"product" + 0.028*"talk" + 0.019*"thinking" + 0.019*"capital" + 0.019*"inventory" + 0.019*"bit" + 0.019*"working" + 0.019*"company"'),
 (4,
  '0.031*"lot" + 0.025*"india" + 0.025*"little" + 0.025*"kind" + 0.019*"product" + 0.019*"bit" + 0.019*"like" + 0.019*"question" + 0.019*"talk" + 0.018*"tim"'),
 (5,
  '0.030*"normal" + 0.024*"bit" + 0.024*"iphone" + 0.018*"market" + 0.018*"growth" + 0.018*"term" + 0.018*"one" + 0.0

In [267]:
import pickle
pickle.dump(data, open('calltranscripts.pic', 'wb'))